## 小說爬蟲

In [23]:
# -*- coding: utf-8 -*
import requests
from bs4 import BeautifulSoup
import types
r = requests.get("https://8book.com/books/novelbook_98524.html")

soup = BeautifulSoup(r.text, "html.parser")

#print(soup)
link = []
for table in soup.find_all("table", {'class' : 'episodelist'}):
    for tr in table.find_all('tr'):
        for a in tr.find_all('a', href=True):
            link.append('https://8book.com' + a['href'])
novel=''
for i in range(len(link)):
    r = requests.get(link[i])
    r.encoding = 'big5'
    soup = BeautifulSoup(r.text, "html.parser")
    st = soup.find('p')
    
    for s in st:
        novel += str(s).replace('(adsbygoogle = window.adsbygoogle || []).push({});', '')
f = open('novel/novel_1.txt','wb')
#f.write(novel.encode('utf-8'))
f.write(novel.replace('<br/>', '').encode('utf-8'))
f.close()

## 切開字詞

In [28]:
import jieba 
import codecs
import os
from argparse import ArgumentParser

def main():
    f = codecs.open('novel/novel_{0}.txt'.format(2), 'r', 'utf-8')
    textseg=[]
    text = ''
    jieba.load_userdict("dict/user_dict_{0}".format(1))
    seg = jieba.cut(f.readlines()[0],cut_all=False)
    for s in seg:
        textseg.append(s)
    fileseg ='segphase/segphases_{0}.txt'.format(2)
    try:
        os.path.isfile(fileseg)
    except:
        open(fileseg,'wb')
    with open(fileseg,'wb') as fW:
        for i in range(len(textseg)):
            fW.write(textseg[i].encode('utf-8'))
            fW.write('\n'.encode('utf-8'))


main()


## word to vector + 文字雲

In [43]:
from gensim.models import word2vec
import jieba
import logging
from argparse import ArgumentParser
from sklearn.decomposition import PCA
from matplotlib import pyplot
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def main():

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    sentences = word2vec.LineSentence('segphase/segphases_{0}.txt'.format(2))
    model = word2vec.Word2Vec(sentences, size=100, alpha=0.0005,min_alpha=0.00001, min_count=20, iter=10)
    model.save("word2vec/word2vec_{0}.model".format(2))
def test():
    model = word2vec.Word2Vec.load('word2vec/word2vec_{0}.model'.format(2))
    phase = input("Please input a phase: ")
    out = model.similar_by_vector(phase,topn=10)
    word = []
    # for i in range(args.topn):
    #     word.append(" ".join(out[i][0]))
    try :
        os.path.isfile('tags.txt')
    except :
        open('tags.txt','wb')
    f = open('tags.txt', 'wb')
    for i in range(10):
        f.write(out[i][0]+" ")
        print(out[i][0])
    f.close()
    # phase2 = input("Please input another phase")
    # print(model.similarity(phase, phase2))
    # X = model[model.wv.vocab]
    # pca = PCA(n_components=2)
    # result = pca.fit_transform(X)

    # pyplot.scatter(result[:, 0], result[:, 1])
    # words = list(model.wv.vocab)
    # for i, word in enumerate(words):
    #     pyplot.annotate(word.encode('utf-8'), xy=(result[i, 0], result[i, 1]))
    # pyplot.show()

    # 讀取每首歌的前10個tags

    text = open('tags.txt').read()
    print(text)
    # 設定停用字(排除常用詞、無法代表特殊意義的字詞)
    # stopwords = {}.fromkeys(["書"])
    # 產生文字雲
    wc = WordCloud(font_path="NotoSerifCJKtc-Black.otf", #設置字體
               background_color="white", #背景顏色
               max_words = 2000 ,)      #停用字詞
    wc.generate(text)
    # 視覺化呈現
    plt.imshow(wc)
    plt.axis("off")
    # plt.figure(figsize=(10,6), dpi = 100)
    plt.show()


if __name__ == '__main__':
    #parser = ArgumentParser()
    #parser.add_argument("-n", "--novel", default=1,type=int)
    #parser.add_argument("-m", "--mode", default='train', type=str)
    #parser.add_argument("-t", "--topn", default=10, type=int)
    #args = parser.parse_args()
    #main()
    test()

2019-06-06 00:57:57,086 : INFO : loading Word2Vec object from word2vec/word2vec_2.model
2019-06-06 00:57:57,087 : WARNING : this function is deprecated, use smart_open.open instead
2019-06-06 00:57:57,160 : INFO : loading wv recursively from word2vec/word2vec_2.model.wv.* with mmap=None
2019-06-06 00:57:57,161 : INFO : setting ignored attribute vectors_norm to None
2019-06-06 00:57:57,161 : INFO : loading vocabulary recursively from word2vec/word2vec_2.model.vocabulary.* with mmap=None
2019-06-06 00:57:57,162 : INFO : loading trainables recursively from word2vec/word2vec_2.model.trainables.* with mmap=None
2019-06-06 00:57:57,163 : INFO : setting ignored attribute cum_table to None
2019-06-06 00:57:57,164 : INFO : loaded word2vec/word2vec_2.model
F:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
2019-06-06 00:57:57,171 : INFO : precomput

ValueError: not enough values to unpack (expected 2, got 0)